In [7]:
import torch 
from deepchopper import  remove_intervals_and_keep_left, smooth_label_region, summary_predict

In [18]:
from deepchopper.utils import alignment_predict, highlight_target, highlight_targets

In [2]:
from pathlib import Path 

In [38]:
cnn_data_folder = "/projects/b1171/ylk4626/project/DeepChopper/logs/eval/runs/2024-04-09_13-56-19/predicts/0/0.pt"
heyna_data_foler = "/projects/b1171/ylk4626/project/DeepChopper/logs/eval/runs/2024-04-09_15-25-53/predicts/0/0.pt"

In [39]:
def test_smooth(data_folder):
    prediction = torch.load(data_folder)
    true_predcition, true_label = summary_predict(prediction['prediction'].argmax(-1).numpy(), prediction['target'].numpy(), -100)
    true_seq, true_label = summary_predict(prediction['seq'].numpy(), prediction['target'].numpy(), -100)
    return true_predcition, true_seq, true_label

In [40]:
cp, cs, cl = test_smooth(cnn_data_folder)

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [41]:
hp, hs, hl = test_smooth(heyna_data_foler)

In [44]:
alignment_predict(cp[0], hp[0])

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

...

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000001111111111110111001111111111111111111111
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:1111111111111111111111111111111111111111100000000000000000000000000000
L:00000000000000000000000000000000000000000000000000000000000

ValueError: zip() argument 2 is shorter than argument 1

In [42]:
alignment_predict(cp[0], cl[0])

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

...

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000

In [43]:
alignment_predict(hp[0], hl[0])

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

...

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:0000000000000000000000000000000000000000000000000000000000000000000000
L:0000000000000000000000000000000000000000000000000000000000000000000000

P:00000000000000000000000000000000000000000000000000000000000
L:00000000000000000000000000000000000000000000000000000000000

In [24]:
smooth_label_region?

Signature:
smooth_label_region(
    labels,
    length_between_intervals_for_merge,
    min_interval_length_threshold,
    min_interval_length_for_discard,
)
Docstring: <no docstring>
Type:      builtin_function_or_method

In [26]:
smooth_label_region(true_predcition[0], 1, 1, 1)

[(403, 404),
 (407, 408),
 (410, 411),
 (413, 414),
 (1749, 1750),
 (2130, 2146),
 (2148, 2211)]

In [28]:
smooth_label_region(true_predcition[0], 1, 1, 2)

[(2130, 2146), (2148, 2211)]